'''
OBJECTIVES:
1. Build WRS system
2. Build Structural BMP Solution evaluator
3. Identify minimum BMP solution front for:
   individual facilities
   facilities w/in departments
   facilities w/in city
   
PYTHON VERSION: 3.6.3  
SQLALCHEMY VERSION: 1.1.13

'''

In [7]:
import winsound

'''
Define basic SQLAlchemy items:
    declarative base object
    connection object
    session object
    DB tables
'''
#SQLAlchemy library items:
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String
from sqlalchemy import update, insert
from sqlalchemy import and_ #used in query.filter() to joing multiple where clauses
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship #http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html#relationship-patterns
from sqlalchemy import inspect

from SQLA_Base import Base #module containing declarative_base
from SQLA_conn_man import session, engine #module handling db and connection creation 

#Table definitions as SQLA classes:
from SQLA_DB_base_bmp_feasibility_test_results import Base_BMP_Feasibility_Test_Results as BBFTR
from SQLA_DB_base_bmp_feasibility_test_definitions import Base_BMP_Feasibility_Test_Definitions as BBFTD
from SQLA_DB_base_bmps import Base_BMPs
from SQLA_DB_combo_bmps import Combo_BMPs
from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
from SQLA_DB_expressions import Expressions
from SQLA_DB_facility_chars import Facility_Chars
from SQLA_DB_facility_monthly_rain import Facility_Monthly_Rain
from SQLA_DB_facility_risks import Facility_Risks
from SQLA_DB_facility_type_has_nel import Facility_Type_Has_NEL
from SQLA_DB_facility_types import Facility_Types
from SQLA_DB_feasibility_test_questions import Feasibility_Test_Questions as FTQ
from SQLA_DB_nel_sample_classes import NEL_Sample_Classes
from SQLA_DB_existing_pollutant_concentrations import Existing_Pollutant_Concentrations as ExPConcs
from SQLA_DB_pollutant_removal_rates import Pollutant_Removal_Rates as PRR
from SQLA_DB_wrs_pollutant_risks import WRS_Pollutant_Risks
Base.metadata.create_all(engine, checkfirst=True) #create SQLA classes

'''
Dictionary of "SQLAlchemy where clause lambda functions" that importCSV uses to test record uniqueness.
used as the where clause in sqlalchemy queries, updates and deletes 
Form:
    {TableName:Lambda Function, TableName:Lambda Function, ...}
    
    TableName is the table name we want to define uniqueness test for
    Lambda Function is a SQLAlchemy query used to test record uniqueness. The function can take on any form 
        but must be made to evaluate the CSV row passed as a dictionary (CSVRowDict in this explanation):
        CSVRowDict: {FieldName:CSVColValue, DBTableFieldName:CSVColValue...} 
            Where: DBTableFieldName is the name of the field associated with the value at CSVColValue on the current row
                   CSVColValue: a value in the CSV's current row+column corresponding to the DBTableFieldName 
        *this assumes that field names are unique across table. if not, then method fails (maybe need to extend method?)
    FALSE: indicates that db table doesn't impose uniqueness on its records (other than its record id being unique)
        
e.g.: lambda myRowVal: Base.metadata.tables['people'].c['name'] == CSVRowDict['name']
        using lambda function in query will search for CSVRowDict's value for 'name' in the table people, field name 
if table has no record uniqueness requirement, then enter: TableName:False
'''
unqTests = {
    'facility_chars': lambda CSVRowDict: Base.metadata.tables['facility_chars'].c['Fac_Name'] == CSVRowDict['Fac_Name'],
    'facility_monthly_rain': False, #DB schema does not impose uniqueness on records in this table
    'facility_type_has_nel': False,
    'facility_risks': False,
    'facility_types': lambda CSVRowDict: Base.metadata.tables['facility_types'].c['Fac_Type'] == CSVRowDict['Fac_Type'],
    'nel_sample_classes': lambda CSVRowDict: Base.metadata.tables['nel_sample_classes'].c['nel_column']==CSVRowDict['NEL_Column'],
    'existing_pollutant_concentrations': False, #uniqueness not imposed for records in this table.
    'wrs_pollutant_risks': False #DB schema does not impose uniqueness on records in this table
}



import SQLA_main as SQLA_main #import main SQLAlchemy functions


In [ ]:
'''
Define other custom modules

'''
import mod_Base_BMP_Eval as BBMP_Eval
import mod_Combo_BMP_Eval as CBMP_Eval
import mod_expression as Expr
import mod_importSpecial as importSpecial #special import functions are defined here
import mod_importCSV as importCSV #generic CSV importer ****IMPORTANT NOTE: function assumes csv in the utf-8-sig file format. weird things happen if its not in this format!!!


In [ ]:
#import feasibillity questions, build feasibility expressions
importSpecial.importFeasibilityQuestionsCSV('Input_Files\\feasibility_test_questions.csv') 

#import base bmp information including:
  #1. imports definitions for cip costs, o&m costs, and BMP sizing to the expressions table
  #2. imports pollutant removal rates into pollutant_removal_rates table
  #3. creates a record in the base_bmps table using (1) and (2)
  #4. feasibility tests
importSpecial.importBaseBMPsCSV('Input_Files\\bmp_lego_piece.csv') 

#IMPORT BASIC FACILITY CHARS:
    #!!!!IMPORTANT!!!! This import must occur before other facility specific data is imported!
print ('\nImporting facility characteristics:')
importCSV.importCSV('Input_Files\\facility_chars.csv', unqTests)

#IMPORT PBP Appendix A1 data
print ('\nImporting PBP Appendix A1 data:')
importCSV.importCSV('Input_Files\\pbp_appxa1.csv', unqTests)

#IMPORT FACILITY RAINFALL EXTRACTED FROM http://rainfall.geography.hawaii.edu/downloads.html
print ('\nImporting Facility Rainfall Data:')
importCSV.importCSV('Input_Files\\FacilityRainfallData.csv', unqTests)

#IMPORT EFFLUENT LIMITS FOR FACILITY TYPES: (either by Priority Based Plan, Table 3 or as City operational assignment)
print ('\nImporting Facility Type Has Effluent Limits:') #import into wrs_pollutant_risks table
importCSV.importCSV('Input_Files\\nel_exists_facility_types.csv', unqTests)

#IMPORT NEL CLASSIFICATION DATA (from PBP Appendix L)
print ('\nImporting NEL Classes')
importCSV.importCSV('Input_Files\\nel_pbp_appxl.csv', unqTests)

#IMPORT FACILITY RISKS:
print ('\nImporting Facility Risks')
#for future implementation:
    #The current process inserts fac risk and update existing_fac_char_id in Facility_chars table. this process thus creates
#dead records. a more sophisticated approach using sophisticated lambda function in unqTests would fix this
importCSV.importCSV('Input_Files\\facility_risks.csv', unqTests)

# #IMPORT FACILITY SAMPLING DATA
 #!!!IMPORTANT!!!! For now, we make none detects = 0 BUT this must be changed to detection limit, per DOH guidance.
print ('\nImporting Facilty Sampling data:')
importCSV.importCSV('Input_Files\\sample_data.csv', unqTests)


# for now, since we're developing, delete out all except 1st 2 facilities.
session.query(ExPConcs).filter(ExPConcs.facility_id >2).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
session.query(Facility_Chars).filter(Facility_Chars.id >2).delete(synchronize_session = False) #http://docs.sqlalchemy.org/en/latest/orm/query.html#sqlalchemy.orm.query.Query.delete
session.commit #we chose not to sync session so need to commit before proceeding to requery or else you may get unpredictable resutls

session.commit()
winsound.Beep(250,1000)

In [ ]:
# EVALUATE BASE BMP FEASIBILITY at each facility
# Write results to the base_bmp_feasibility_test_results table.

print('\n******Evaluating Base BMP feasibility at facilities.******')
Expr.ResetEvalErrorCount() #RESET EXPRESION EVALUATOR ERROR COUNT
for aFac in session.query(Facility_Chars):
    print ('\n***Evaluating base bmp feasibiilty tests for facility: ', aFac.Fac_Name), ' ***'
    myBMPs = session.query(Base_BMPs)
    for aBMP in myBMPs:
        print ('\nEvaluating feasibility of base_bmp: ', aBMP.bmp_name, ' ID: ', aBMP.id)
        BBMP_Eval.Eval_base_bmp_feasibility_tests(aFac, aBMP)
session.commit
winsound.Beep(250,1000)
print ('*****************************************************************')
print ('* Completed evaluating Base BMP feasibility                     *')
if Expr.CountEvalErrors() >0:
    print (Expr.CountEvalErrors(), ' errors were encountered. Review output to identify location(s)')
    print ('Hint: expression evaluation error lines are prefixed by: FAULT!!!! Error occured while evaluating expression:')
else:
    print ('No errors detected.')
print ('*****************************************************************')

In [ ]:
#Estimate Pollutant Effluent Limits
#Differentiate between wet and dry season limits (this only applies to NEL Column I vs II; I is dry season, II is wet season)

#NEED TO FIX SCHEMA: RIGHT NOW WE WRITE NEL_COLUMN FROM PBP_APPXA1.CSV INTO FAC_CHARS FIELD: NEL_COLUMN.
#BUT; WE ALSO HAVE A FIELD IN FACILITY_RISKS FOR WET AND DRY SEASON NEL COLUMNS.
#WE SHOULD USE THE FACILITY_RISKS BY MAKING A SEPARATE WET AND DRY SEASON FIELD
#OR WE SHOULD USE THE FAC_RISKS FIELDS.

#I THINK WE SHOULD PLACE INTO FAC_CHARS TO FURTHER CLARIFY THAT NELS ARE NOT A FACILITY RISK

#FOR EACH FACILITY, USE ITS SAMPLE CLASS ASSIGNMENT TO 

for fac in session.query(Facility_Chars):
    print (fac.NEL_Column)


In [ ]:
#CREATE COMBO BMPS USING BASE BMPS
#ALL POSSIBLE COMBOS WILL BE CREATED AND ADDED TO THE COMBO_BMPS TABLE
#MAXIMUM POLLUTANT REMOVAL RATES ARE DETERMINED BY IDENTIFYING 
#  THE BASE_BMP IN THE COMBO THAT PROVIDES THE HIGHEST REMOVAL RATE FOR A GIVEN POLLUTANT

import time
print ('get a coffee...this one takes a while!')
start_time = time.time()
CBMP_Eval.Make_ALL_bmp_base_option_combos()
session.commit()
print ('--- %s execution time in seconds ---' % (time.time() - start_time))

In [ ]:
'''
Identify the feasible bmp combinations for each facility
Use base bmp feasibility results for each facility.
Put results into the combo_bmp_feasibility_test_results table
'''
import itertools     #https://docs.python.org/3/library/itertools.html    
import pandas as pd

from sqlalchemy import and_

def _Make_bmp_fingerprint(base_BMP_components):
    #create fingerprint of the passed list of base_bmp_ids
    #fingerprint is just a | separated list of ids of the base bmps that make up the combo bmp
    #corresponds to bmp_options table's bmp_fingerprint field
    #FORMAT: |bmp_option_base_component_id||bmp_option_base_component_id| w/ id's given in ascending order
    fingerprint = '|' + '|'.join(str(id) + '|' for id in base_BMP_components)
    return fingerprint

def Eval_FacBMPCombo(pd_basebmps, myFacility, bmpCombo):
    '''
    input:
        pdbasebmps: pandas built from a BBMP_Eval.evalFacility_BaseBMP dictionary list
                    assme that pandas is passed in w/ index is set as base_bmp_id
        myFacility: SQLA fac_chars record
        bmpCombo: list of base_bmp_ids that make up this combo
    
    #retrieve previously computed combo removal rate
    #calculate combo cip and om cost, insert/update database
    #calculate wrs reduction, insert/update database

    #return as pandas    
    '''    
    #get combo bmp pollutant removal rates into pandas 
    q = session.query(Combo_BMPs.bmp_fingerprint, Combo_BMPs.id.label('combos_bmp_id'), PRR.id.label('PRR_id'),
          PRR.r_tss, PRR.r_turbidity, PRR.r_p, PRR.r_n, PRR.r_nn, PRR.r_an,
          PRR.r_og, PRR.r_cu, PRR.r_zn, PRR.r_fe, PRR.r_phmin, PRR.r_phmax
        ).filter(Combo_BMPs.bmp_fingerprint == _Make_bmp_fingerprint(bmpCombo)).filter(
        Combo_BMPs.bmp_option_removal_rate_id == PRR.id)  
    pd_rr = pd.read_sql(q.statement,session.bind) 

    #use information in pd_rr to get CBFTR_record - make new record if necessary
    myCBFTR = Base.metadata.tables['combo_bmp_feasibility_test_results']
    myCBFTR_id = SQLA_main.insertupdateRec(myCBFTR,{'facility_id':myFacility.id, 'combo_bmps_id':pd_rr['combos_bmp_id'][0]},
                and_(
        myCBFTR.c['facility_id'] == myFacility.id,
        myCBFTR.c['combo_bmps_id'] == pd_rr['combos_bmp_id'][0]
                    ))
    session.flush()
    
    print (myCBFTR_id)
    
    #calculate WRS reduction
#     myFac_exWRSData = session.query(
    
    

    #get costs in pandas
    sumCIP = sum(pd_basebmps.loc[bmp_id,'calc_cip_cost'] for bmp_id in bmpCombo)
    sumOM = sum(pd_basebmps.loc[bmp_id,'calc_om_cost'] for bmp_id in bmpCombo)
    pd_sums = pd.DataFrame([{'calc_cip_cost':sumCIP, 'calc_om_cost': sumOM}])

    #merge combo bmp's removal rates and costs into 1 dataframe
    return pd.concat([pd_rr, pd_sums], axis = 1)
    
    

def Eval_FacBMPOptions(myFacility):
    #a wrapper around Eval_FacBMPCombo
    print('\n***Evaluating feasible bmp combos for facility: ', aFac.Fac_Name, '***')
    print ('****Evaluating feasibile base bmps****')
    df = pd.DataFrame(BBMP_Eval.evalFacility_BaseBMP(aFac, False)).set_index('base_bmp_id')
    display (df)   
    print ('****These are the feasible base bmps. I\'ll use them to make combos:****')
    df = df.loc[df['is_feasible'] == 1]
    display (df)
    feas_ls = df.index #send feasible base bmp ids to list
#     print (feas_ls)
# from SQLA_DB_combo_bmps import Combo_BMPs
# from SQLA_DB_combo_bmp_feasibility_test_results import Combo_BMP_Feasibility_Test_Results as CBFTR
    for CBOLen in range (1, len(feas_ls)+1): #+1 so it's inclusive of last count
        for combo in  itertools.combinations(feas_ls,CBOLen):
            print ('Here is a summary of the combo: ', list(combo))
            display(Eval_FacBMPCombo(df,myFacility, list(combo)))

            
def Eval_All_FacBMPOptions():
    print ('Evaluating feasibile BMP Options for each facility:')
    for aFac in session.query(Facility_Chars):
        Eval_FacBMPOptions(aFac)
    
Eval_All_FacBMPOptions()
session.commit()

In [ ]:
# session.close()
# engine.dispose()

In [ ]:
# http://pythonhow.com/accessing-dataframe-columns-rows-and-cells/
import pandas as pd #import in pandas library
print ('#get csv data and read into pandas')
df1=pd.read_csv("http://pythonhow.com/wp-content/uploads/2016/01/Income_data.csv")
print (df1)
print ('#write new dataframe w/ index set to the "State" column in the csv')
df2=df1.set_index("State").copy()
print (df2)
print ('#extract a portion of the dataframe: States = Alaska to Arkansas; and Dates 2005:2007')
print (df2.loc["Alaska":"Arkansas","2005":"2007"])

print ('Get only certain States, using a list of states:')
getStates = ['Alaska', 'Arizona']
print (df2.loc[getStates])

print ('#slice a column:')
df2.loc[: , "2005"]
print ('get a cell:')
df2.loc['Alaska','2005']
print ('#get max of 2005 data')
print (df2.loc[:,'2005'].max())
print ('take 2005 column and put into list')
LS = df2['2005'].tolist() #this is a series. we use the .tolist() to convert from series to list
print (type(LS))
